# Hyperparameters

In [ ]:
!pip install -q matplotlib

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
class HParam():
    def __init__(self):
        self.channels = 3
        self.image_max_size = 100
        self.image_size = 32
        self.input_shape = (self.image_size, self.image_size, self.channels)
        
        self.shuffle_buffer = 10000
        self.batch_size = 1024
        self.valid_size = 3000
        
        self.epochs = 100
        self.steps_per_epoch = 10

hparam = HParam()

# Data Preparation

There are 3 methods to resize a image in tensorflow:
1. resize: arbitrary resize image to given size, do not preserve aspect ratio
2. resize_with_pad: preserve aspect ratio, resize image to given size, do not scale proportionally
3. resize_with_crop_or_pad: do not resize image, simply central crop or margin pad to given size

To resize image proportionally, the idea is that we first pad all images to a bigger size, and then resize them down to a smaller desiered size.

In [ ]:
N_DEGREE = 4
N_LABEL = 6

def parse_image(image):
    image = tf.io.decode_png(image, hparam.channels)
    image = tf.image.resize_with_pad(image, hparam.image_size, hparam.image_size)
    return image

def parse_image_proportional(image):
    image = tf.io.decode_png(image, hparam.channels)
    image = tf.image.resize_with_crop_or_pad(image, hparam.image_max_size, hparam.image_max_size)
    image = tf.image.resize(image, (hparam.image_size, hparam.image_size))
    return image

def parse_single_example(example_proto, img_parse_fn):
    feature_spec = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "degree": tf.io.FixedLenFeature([], tf.int64),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    features = tf.io.parse_single_example(example_proto, feature_spec)

    features["image"] = img_parse_fn(features["image"])
    features["degree"] = tf.one_hot(features["degree"], N_DEGREE)
    features["label"] = tf.one_hot(features["label"], N_LABEL)

    label = features["label"]
    return features, label

tfrecord_paths = [
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/NG-InversePolarity.tfrecord',
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/NG-MoreComp.tfrecord',
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/NG-NoneComp.tfrecord',
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/NG-OutsidePosition.tfrecord',
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/NG-UpsideDown.tfrecord',
    '/data/aoi-wzs-p3-dip-prewave-saiap/experiments/adversarial-training/OK.tfrecord',
]
datasets = [tf.data.TFRecordDataset(x).repeat() for x in tfrecord_paths]
dataset = tf.data.experimental.sample_from_datasets(datasets)

## plot parsed image

In [ ]:
def plot_image(img_parse_fn):
    n_col = 5
    for x, _ in dataset.map(lambda x: parse_single_example(x, img_parse_fn), tf.data.experimental.AUTOTUNE).batch(n_col**2).take(1):
        pass
    x["image"] /= 255.

    fig, axes = plt.subplots(n_col, n_col, sharex=True, sharey=True, figsize=(10, 10))
    for ax, im in zip(axes.flatten(), x["image"]):
        ax.imshow(im)
        ax.axis("off")

In [ ]:
plot_image(parse_image)

In [ ]:
plot_image(parse_image_proportional)

# Base Model

In [ ]:
import tensorflow.keras.layers as l

def BaseModel():
    inputs = {
        "image": tf.keras.Input(hparam.input_shape, name="image"),
        "degree": tf.keras.Input([N_DEGREE], name="degree")
    }
    e = inputs["degree"]
    
    x = inputs["image"]
    x = l.Conv2D(64, 7, padding="same")(x)
    x = l.ReLU()(x)
    for filters in [128, 256, 512]:
        x = l.Conv2D(filters, 3, padding="same")(x)
        x = l.BatchNormalization()(x)
        x = l.ReLU()(x)
        x = l.MaxPool2D()(x)
    x = l.Conv2D(1024, 3, padding="same")(x)
    x = l.BatchNormalization()(x)
    x = l.ReLU()(x)
    x = l.GlobalAveragePooling2D()(x)
    x = l.Concatenate()([x, e])
    x = l.Dense(N_LABEL)(x)
    output = l.Activation("softmax", dtype="float32")(x)
    return tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
metrics = [
    tf.keras.metrics.Recall(name="recall/NG-InversePolarity", class_id=0),
    tf.keras.metrics.Recall(name="recall/NG-MoreComp", class_id=1),
    tf.keras.metrics.Recall(name="recall/NG-NoneComp", class_id=2),
    tf.keras.metrics.Recall(name="recall/NG-OutsidePosition", class_id=3),
    tf.keras.metrics.Recall(name="recall/NG-UpsideDown", class_id=4),
    tf.keras.metrics.Recall(name="recall/OK", class_id=5),
    tf.keras.metrics.Precision(name="precision/NG-InversePolarity", class_id=0),
    tf.keras.metrics.Precision(name="precision/NG-MoreComp", class_id=1),
    tf.keras.metrics.Precision(name="precision/NG-NoneComp", class_id=2),
    tf.keras.metrics.Precision(name="precision/NG-OutsidePosition", class_id=3),
    tf.keras.metrics.Precision(name="precision/NG-UpsideDown", class_id=4),
    tf.keras.metrics.Precision(name="precision/OK", class_id=5),
]

# parse_image

In [ ]:
train_ds = dataset.skip(hparam.valid_size).shuffle(hparam.shuffle_buffer).map(lambda x: parse_single_example(x, parse_image), tf.data.experimental.AUTOTUNE).batch(hparam.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_ds = dataset.take(hparam.valid_size).shuffle(hparam.shuffle_buffer).map(lambda x: parse_single_example(x, parse_image), tf.data.experimental.AUTOTUNE).batch(hparam.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
logdir = "/data/aoi-wzs-p3-dip-prewave-saiap/experiments/resize-proportional/parse_image"
callbacks = [
    tf.keras.callbacks.TensorBoard(logdir, write_graph=False, profile_batch=0)
]

model = BaseModel()
model.compile("adam", "categorical_crossentropy", metrics=metrics)

model.fit(train_ds, validation_data=valid_ds,
          epochs=hparam.epochs, steps_per_epoch=hparam.steps_per_epoch,
          callbacks=callbacks,
          verbose=0)

# parse_image_proportional

In [ ]:
train_ds = dataset.skip(hparam.valid_size).shuffle(hparam.shuffle_buffer).map(lambda x: parse_single_example(x, parse_image_proportional), tf.data.experimental.AUTOTUNE).batch(hparam.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_ds = dataset.take(hparam.valid_size).shuffle(hparam.shuffle_buffer).map(lambda x: parse_single_example(x, parse_image_proportional), tf.data.experimental.AUTOTUNE).batch(hparam.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
logdir = "/data/aoi-wzs-p3-dip-prewave-saiap/experiments/resize-proportional/parse_image_proportional"
callbacks = [
    tf.keras.callbacks.TensorBoard(logdir, write_graph=False, profile_batch=0)
]

model = BaseModel()
model.compile("adam", "categorical_crossentropy", metrics=metrics)

model.fit(train_ds, validation_data=valid_ds,
          epochs=hparam.epochs, steps_per_epoch=hparam.steps_per_epoch,
          callbacks=callbacks,
          verbose=0)